In [3]:
import os
import json
import cv2

# ディレクトリのパス設定
pose_dir = "../../ground_truth/Indoor/pose"
video_dir = "../../videos/Indoor"
output_dir = "../../videos/Indoor/GT_drawn"
os.makedirs(output_dir, exist_ok=True)

# BGR表記の色定義（各値に0.8を乗算）
colors = {
    "Red_Jersey": (0, 0, 204),
    "Blue_Jersey": (204, 0, 0),
    "Pink_Jersey": (144, 84, 204),
    "Black_Jersey": (0, 0, 0),
    "Orange_Jersey": (0, 132, 204),
    "Green_Jersey": (108, 204, 176),
}

# MOT用のカラー変換マッピング（lowercase）
mot_color_map = {
    "red": (0, 0, 204),
    "blue": (204, 0, 0),
    "pink": (144, 84, 204),
    "black": (0, 0, 0),
    "orange": (0, 132, 204),
    "green": (108, 204, 176),  # yellowはgreenとして解釈
}

# pose_dir内の全JSONファイルを処理
for file in os.listdir(pose_dir):
    if not file.endswith(".json"):
        continue

    json_path = os.path.join(pose_dir, file)
    with open(json_path, "r") as f:
        data = json.load(f)

    # JSON内の動画情報を取得
    video_info = data.get("video", {})
    video_filename = video_info.get("file_name")
    if video_filename is None:
        print(f"{file} には動画ファイル名の情報がありません。")
        continue

    # 入力動画のパスを構築
    video_path = os.path.join(video_dir, video_filename)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"動画 {video_path} をオープンできませんでした。")
        continue

    # 動画情報の取得
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video_path = os.path.join(output_dir, video_filename)
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # --- MOTファイル（bboxの座標情報）の読み込み ---
    # 動画名が "video1.mp4" なら MOTファイルは "video1.txt" と仮定
    mot_filename = os.path.splitext(video_filename)[0] + ".txt"
    mot_file_path = os.path.join("../../ground_truth/Indoor/MOT_files", mot_filename)
    mot_bbox = {}
    if os.path.exists(mot_file_path):
        with open(mot_file_path, "r") as f:
            lines = f.readlines()
        for line in lines:
            parts = line.strip().split(",")
            if len(parts) < 6:
                continue
            # 一般にMOTは1-indexedとするので、フレーム番号を0-indexedに変換
            frame_num = int(parts[0]) - 1
            mot_id = int(parts[1])
            x = float(parts[2])
            y = float(parts[3])
            w = float(parts[4])
            h = float(parts[5])
            mot_bbox.setdefault(frame_num, []).append((mot_id, x, y, w, h))
    else:
        print("MOTファイルが見つかりません:", mot_file_path)

    # --- カラー情報ファイルの読み込み ---
    # transformed_MOT内のファイル名は "video1_color.txt" のように仮定
    base_name = os.path.splitext(video_filename)[0]
    color_file_name = base_name + "_color.txt"
    color_file_path = os.path.join("../../ground_truth/Indoor/transformed_MOT", color_file_name)
    color_info = {}
    if os.path.exists(color_file_path):
        with open(color_file_path, "r") as f:
            lines = f.readlines()
        for line in lines:
            parts = line.strip().split(",")
            if len(parts) < 5:
                continue
            frame_num = int(parts[0]) - 1  # 1-indexed → 0-indexed
            mot_id = int(parts[1])
            # parts[2], parts[3]は座標情報（ここでは利用せず色のみ利用）
            col = parts[4].strip().lower()
            # yellowはgreenに変換
            if col == "yellow":
                col = "green"
            color_info[(frame_num, mot_id)] = col
    else:
        print("カラー情報ファイルが見つかりません:", color_file_path)

    annotations = data.get("annotations", {})
    frame_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        key = f"frame_{frame_index}"
        if key in annotations:
            for jersey, points in annotations[key].items():
                # 指定色がない場合は緑色をデフォルトとする
                color = colors.get(jersey, (0, 255, 0))
                for pt in points:
                    # 座標を整数に変換して描画
                    center = (int(pt[0]), int(pt[1]))
                    cv2.circle(frame, center, radius=4, color=color, thickness=-1)

        # --- MOTのbbox描画 ---
        if frame_index in mot_bbox:
            for bbox in mot_bbox[frame_index]:
                mot_id, x, y, w, h = bbox
                # カラー情報ファイルから色を取得（なければデフォルトの緑）
                col_str = color_info.get((frame_index, mot_id), None)
                if col_str is None:
                    bgr = (0, 255, 0)
                else:
                    bgr = mot_color_map.get(col_str, (0, 255, 0))
                # bboxの描画（整数化）
                top_left = (int(x), int(y))
                bottom_right = (int(x + w), int(y + h))
                cv2.rectangle(frame, top_left, bottom_right, color=bgr, thickness=2)

        out.write(frame)
        frame_index += 1

    cap.release()
    out.release()
    print(f"動画 {video_filename} の描画処理が完了し、{output_video_path} に保存されました。")


動画 basket_S6T3_post.mp4 の描画処理が完了し、../../videos/Indoor/GT_drawn/basket_S6T3_post.mp4 に保存されました。
動画 basket_S4T4_post.mp4 の描画処理が完了し、../../videos/Indoor/GT_drawn/basket_S4T4_post.mp4 に保存されました。
動画 basket_S6T2_post.mp4 の描画処理が完了し、../../videos/Indoor/GT_drawn/basket_S6T2_post.mp4 に保存されました。
動画 basket_S5T5_post.mp4 の描画処理が完了し、../../videos/Indoor/GT_drawn/basket_S5T5_post.mp4 に保存されました。
動画 basket_S6T7_post.mp4 の描画処理が完了し、../../videos/Indoor/GT_drawn/basket_S6T7_post.mp4 に保存されました。
動画 basket_S4T7_post.mp4 の描画処理が完了し、../../videos/Indoor/GT_drawn/basket_S4T7_post.mp4 に保存されました。
動画 basket_S6T4_post.mp4 の描画処理が完了し、../../videos/Indoor/GT_drawn/basket_S6T4_post.mp4 に保存されました。
動画 basket_S6T6_post.mp4 の描画処理が完了し、../../videos/Indoor/GT_drawn/basket_S6T6_post.mp4 に保存されました。
動画 basket_S5T7_post.mp4 の描画処理が完了し、../../videos/Indoor/GT_drawn/basket_S5T7_post.mp4 に保存されました。
動画 basket_S6T1_post.mp4 の描画処理が完了し、../../videos/Indoor/GT_drawn/basket_S6T1_post.mp4 に保存されました。
動画 basket_S6T5_post.mp4 の描画処理が完了し、../../videos/Indoor/GT_dra